### Intersection of mrcanavar, PacBio CCS, and ONT high coverage regions to identify potential CNV

Generating excessive coverage bed files using calculations from mosdepth for both PacBio CCS 15kb_20kb merged and ONT bam files.

HG005vGRCh37_wm_ONT.sort.bam is from T2T Finishing Workshop globus /team-variants/read_aligns/hg005_grch37/HG005vGRCh37_wm_ONT.sort.bam

HG005.hs37d5.haplotagged_w_10x_reformatted.bam is from https://drive.google.com/file/d/10wrFuGzyjwLWoP-MYWPVKPeL7jTZLaOJ/view?usp=sharing


HG005.hs37d5.300x.bam.bilkentuniv.052119.dups.bed is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/ChineseTrio/analysis/BilkentUni_IlluminaHiSeq_TARDIS_mrCaNaVar_05212019/Chinese-trio-HG005.hs37d5.300x.bam.bilkentuniv.052119.dups.bed.gz

HG005.hs37d5.pbsv.vcf.gz is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/ChineseTrio/analysis/PacBio_CCS_15kb_20kb_chemistry2_12072020/HG005/HG005.hs37d5.pbsv.vcf.gz

convert_mosdepth_to_excessive_coverage.py is at the end of this notebook

In [ ]:
### mosdepth commands

In [ ]:
mosdepth -b 1000 -x --mapq 20 --no-per-base HG005.hs37d5.haplotagged_w_10x_reformatted_window_size_1000_MAPQ20 HG005.hs37d5.haplotagged_w_10x_reformatted.bam



samtools view -1 -F 0x100 HG005vGRCh37_wm_ONT.sort.bam -h > HG005_GRCh37_ONT_filtered.bam

mosdepth -b 1000 -x --no-per-base HG005_GRCh37_ONT_filtered_window_size HG005_GRCh37_ONT_filtered.bam

In [ ]:
### Find coverage levels of excessive coverage on CCS and ONT data in R

In [ ]:
chr_1_22 <- c("1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22")
mosdepth_PB_HiFi_1000_window_size = read.delim("HG005.hs37d5.haplotagged_window_size_1000_MAPQ20.regions.bed", col.names = c("CHR","START","END","DEPTH"))
mosdepth_PB_HiFi_1000_window_size_chr_1_22 <- mosdepth_PB_HiFi_1000_window_size[which(mosdepth_PB_HiFi_1000_window_size[,"CHR"] %in% chr_1_22),] 
quantile(mosdepth_PB_HiFi_1000_window_size_chr_1_22[,"DEPTH"])


IQR(mosdepth_PB_HiFi_1000_window_size_chr_1_22[,"DEPTH"])
#: 12.48
(quantile(mosdepth_PB_HiFi_1000_window_size_chr_1_22[,"DEPTH"])[3]/2)*2.5
#: 59.4875 

mosdepth_ONT_1000_window_size = read.delim("HG005_GRCh37_ONT_filtered_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))
mosdepth_ONT_1000_window_size_chr_1_22 <- mosdepth_ONT_1000_window_size[which(mosdepth_ONT_1000_window_size[,"CHR"] %in% chr_1_22),]

quantile(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])

IQR(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])
#: 12.32
(quantile(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])[3]/2)*2.5
#: 71.6875

In [ ]:
### HG005_mrcanavar_intersect_ccs_window_size_cnv_threshold_intersect_ont_window_size_cnv_threshold.bed

### What this does: find potential CNVs in HG4 from intersecting coverage files from PacBio HiFi, ONT, and Illumina data. This generates PacBio HiFi excessive coverage bed, intersect with mrCaNaVar dups bed, generates ONT excessive coverage bed, intersects to these all to generate exclusion bed

In [ ]:
python convert_mosdepth_to_excessive_coverage.py --input HG005.hs37d5.haplotagged_window_size_1000_MAPQ20.regions.bed --output HG005.hs37d5.haplotagged_window_size_1000_window_size_excessive_coverage_cnv_threshold.bed --threshold 59.4875 

bedtools intersect -a Chinese-trio-HG005.hs37d5.300x.bam.bilkentuniv.052119.dups.bed -b HG005.hs37d5.haplotagged_window_size_1000_window_size_excessive_coverage_cnv_threshold.bed > mrcanavar_intersect_PacBio_HiFi_cnv_threshold.bed

python convert_mosdepth_to_excessive_coverage.py --input HG005_GRCh37_ONT_filtered_window_size.regions.bed --output HG005_GRCh37_ONT_filtered_1000_window_size_excessive_coverage_cnv_threshold.bed --threshold 71.6875

bedtools intersect -a mrcanavar_intersect_PacBio_HiFi_cnv_threshold.bed -b HG005_GRCh37_ONT_filtered_1000_window_size_excessive_coverage_cnv_threshold.bed > mrcanavar_intersect_PacBio_HiFi_cnv_threshold_intersect_ONT_cnv_threshold.bed


In [ ]:
### HG005_GRCh37_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_outlier_threshold.bed

### What this does: Find another set of potential CNVs from computing a coverage threshold using an elliciptal outlier for PacBio HiFi and ONT CNV file steps in R

mosdepth_PacBio_HiFi_1000_window_size = read.delim("HG005.hs37d5.haplotagged_window_size_1000_MAPQ20.regions.bed", col.names = c("CHR","START","END","DEPTH"))

mosdepth_ONT_1000_window_size = read.delim("HG005_GRCh37_ONT_filtered_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))

chr_1_22 <- c("1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22")

mosdepth_PacBio_HiFi_1000_window_size_chr_1_22 <- mosdepth_PacBio_HiFi_1000_window_size[which(mosdepth_PacBio_HiFi_1000_window_size[,"CHR"] %in% chr_1_22),]

mosdepth_ONT_1000_window_size_chr_1_22 <- mosdepth_ONT_1000_window_size[which(mosdepth_ONT_1000_window_size[,"CHR"] %in% chr_1_22),]

df_mosdepth_PacBio_HiFi_1000_window_size <- data.frame(mosdepth_PacBio_HiFi_1000_window_size_chr_1_22) 

df_mosdepth_ONT_1000_window_size <- data.frame(mosdepth_ONT_1000_window_size_chr_1_22)

df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined <- df_mosdepth_PacBio_HiFi_1000_window_size

df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined[,5] <- mosdepth_ONT_1000_window_size_chr_1_22[,4]

colnames(df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined) <- c("CHR", "START", "END", "CCS_DEPTH", "ONT_DEPTH")

df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_values <- df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined 

medianccsdepth = median(mosdepth_PacBio_HiFi_1000_window_size_chr_1_22[,"DEPTH"]) 

medianontdepth = median(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"]) 

df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_values[,6] <- sqrt(((df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined[,4]/medianccsdepth)^2 + (df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined[,5]/medianontdepth)^2)/2) 

threshold_ellipctial_outlier = unname(quantile(df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_values[,6])[4]+(1.5*IQR(df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_values[,6])[1]))

df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_outliers <- df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_values[which(df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_values[,6] > threshold_ellipctial_outlier),]

write.csv(df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_outliers, file = "HG005_GRCh37_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_outlier_threshold.bed", row.names = FALSE)


In [ ]:
## GRCh37 generate SV exclusion bed taking union of >=50bp SVs from dipcall, svanalyzer, and pbsv

HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns.vcf and HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns.vcf from Nancy Hansen on T2T globus


HG005.hs37d5.pbsv.vcf from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/ChineseTrio/analysis/PacBio_CCS_15kb_20kb_chemistry2_12072020/HG005/HG005.hs37d5.pbsv.vcf.gz


HG005_hifiasm-align2-GRCh37.dip.vcf from https://drive.google.com/drive/folders/1wLskuFUgd8KoQY4CYgb2ANUPFRoTA0RM

In [ ]:
python convert_vcf_to_bed_svanalyzer_gt_50bp.py --input HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns.vcf --output HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp.bed

python convert_vcf_to_bed_svanalyzer_gt_50bp.py --input HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns.vcf --output HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp.bed

python convert_vcf_to_bed_pbsv.py --input HG005.hs37d5.pbsv.vcf --output HG005.hs37d5.pbsv.bed

python convert_vcf_to_bed_dipcall.py --input HG005_hifiasm-align2-GRCh37.dip.vcf --output HG005_GRCh37_hifiasm_gt50bp.bed


cat HG005_GRCh37_hifiasm_gt50bp.bed | sort -k1,1 -k2,2n > HG005_GRCh37_hifiasm_gt50bp_sorted.bed

cat HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp.bed | sort -k1,1 -k2,2n > HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted.bed

cat HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp.bed | sort -k1,1 -k2,2n > HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted.bed

cat HG005_GRCh37_hifiasm_gt50bp.bed | sort -k1,1 -k2,2n > HG005_GRCh37_hifiasm_gt50bp.bed


python remove_all_but_chr1_22.py --input HG005_GRCh37_hifiasm_gt50bp_sorted.bed --output HG005_GRCh37_hifiasm_gt50bp_sorted_chr1_22.bed

python remove_all_but_chr1_22.py --input HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted.bed --output HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted_chr1_22.bed

python remove_all_but_chr1_22.py --input HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted.bed --output HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted_chr1_22.bed

python remove_all_but_chr1_22.py --input HG005.hs37d5.pbsv_gt50bp_sorted.bed --output HG005.hs37d5.pbsv_gt50bp_sorted_chr1_22.bed


multiIntersectBed -i HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted_chr1_22.bed HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted_chr1_22.bed HG005.hs37d5.pbsv_gt50bp_sorted_chr1_22.bed HG005_GRCh37_hifiasm_gt50bp_sorted_chr1_22.bed | mergeBed -i stdin > HG005_GRCh37_union_SVs.bed

cat HG005_GRCh37_union_SVs.bed | awk '{sum+=$3-$2} END {print sum}'


cat GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed | sort -k1,1 -k2,2n > GRCh37_AllTandemRepeatsandHomopolymers_slop5_sorted.bed

intersectBed -wa -a GRCh37_AllTandemRepeatsandHomopolymers_slop5_sorted.bed -b HG005_GRCh37_union_SVs.bed | multiIntersectBed -i stdin HG005_GRCh37_union_SVs.bed | awk '{FS=OFS="\t"} {print $1,$2-50,$3+50}' | mergeBed -i stdin -d 1000 > HG005_GRCh37_union_SVs_repeatexpanded_slop50_merge1000.bed

cat HG005_GRCh37_union_SVs_repeatexpanded_slop50_merge1000.bed | awk '{sum+=$3-$2} END {print sum}'


bedtools slop -i HG005_GRCh37_union_SVs_repeatexpanded_slop50_merge1000.bed -g human.b37.genome -b 0.25 -pct | sort -k1,1 -k2,2n | mergeBed -i stdin > HG005_GRCh37_union_SVs_repeatexpanded_slop50_merge1000_expanded_150_percent.bed

cat HG005_GRCh37_union_SVs_repeatexpanded_slop50_merge1000_expanded_150_percent.bed | awk '{sum+=$3-$2} END {print sum}'

In [ ]:
### Inversions 

Nancy Hansen provided SVanalyzer calls and extracted inversions that were identified in HG005 in January 2020

In [ ]:
grep -e "^#" -e "INV" HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns.vcf > HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS.vcf

grep -e "^#" -e "INV" HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns.vcf > HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS.vcf

python convert_inversions_vcf_to_bed.py --input HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS.vcf --output HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened.bed

python convert_inversions_vcf_to_bed.py --input HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS.vcf --output HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened.bed

cat HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened.bed | sort -k1,1 -k2,2n > HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened_sorted.bed

cat HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened.bed | sort -k1,1 -k2,2n > HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened_sorted.bed

multiIntersectBed -i HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened_sorted.bed HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened_sorted.bed > HG005_inversions.bed


bedtools slop -i HG005_inversions.bed -g human.b37.genome -b 0.25 -pct > HG005_inversions_slop150.bed

python remove_all_but_chr1_22.py --input HG005_inversions_slop150.bed --output HG005_inversions_slop150_chr1_22.bed

#### convert_mosdepth_to_excessive_coverage.py script

import argparse

parser = argparse.ArgumentParser(description="Subset bed file to callable regions only")
parser.add_argument('--input_file', metavar="I", type=str, nargs="+", help="input bed file")
parser.add_argument('--output_file', metavar="O", type=str, nargs="+", help="output file")
parser.add_argument('--threshold', metavar="T", type=str, nargs="+", help="input threshold")
args = parser.parse_args()

f = open(args.input_file[0], "r") 
f_lines = f.readlines()

f_out = open(args.output_file[0], "w+")
threshold = float(args.threshold[0])

for line in f_lines:   
    if "DEPTH" in line: 
        continue
    line_split = line.split("\t")
    depth_field = float(line_split[3])
    if depth_field > threshold:
        f_out.write(line)
        f_out.flush()  

f.close()
f_out.close()

remove_all_but_chr1_22.py

import argparse
parser = argparse.ArgumentParser(description="Remove all but chr1-22 entries") 
parser.add_argument('--input', metavar="I", type=str, nargs="+", help="input file") 
parser.add_argument('--output', metavar="O", type=str, nargs="+", help="output file") 
args = parser.parse_args()

f = open(args.input[0], "r") 
f_lines = f.readlines() 
chrs_to_keep = ["chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr19","chr20","chr21","chr22"]

f_out = open(args.output[0], "w+")

for line in f_lines: 
    split_line = line.split("\t") 
    if split_line[0] in chrs_to_keep: 
        f_out.write(line)
        f_out.flush()
        
f.close()
f_out.close()
